In [9]:
import numpy as np
from tqdm import tqdm
from itertools import combinations
import pandas as pd
from scipy.sparse import csr_matrix
from multiprocessing import Pool
from recsys.data.dataset import Dataset
from recsys.io.file import IOService

In [ ]:
FILEPATH = "data/dev/ratings_0.5_pct.pkl"

In [ ]:
ratings = Dataset(filepath=FILEPATH)


In [ ]:
ratings.info()

In [ ]:
ratings.summarize()

In [ ]:
def to_key(uv: tuple) -> str:
    return str(uv[0]) +"_" + str(uv[1])
def from_key(uv: str) -> tuple:
    return tuple([int(x) for x in uv.split("_")])

In [ ]:
UV = {}

for item in tqdm(ratings.items):
    item_ratings = ratings.get_item_ratings(item=item)
    for uv_pair in combinations(item_ratings['userId'].values, 2):
        uv_key = to_key(uv_pair)
        if UV.get(uv_key,None) is not None:
            UV[uv_key].append(item)
        else:
            UV[uv_key] = [item]


In [ ]:
print(f"There are {len(UV)} distinct associated user pairs")

In [ ]:
def cos(uv, items):
    u,v = from_key(uv)
    ru = ratings.get_users_items_ratings(items=items, users=[u])
    rv = ratings.get_users_items_ratings(items=items, users=[v])
    l2u = ratings.get_user_rating_norms(user=u)
    l2v = ratings.get_user_rating_norms(user=v)
    sim = ru.dot(rv) / (l2u * l2v)
    sim = {'u': u, 'v': v, 'sim': sim}
    return sim

In [ ]:
S = {}
for uv, uv_ratings in tqdm(I.items()):
    u, v = from_key(uv)
    rui = uv_ratings[uv_ratings["userId"] == u].sort_values(by="movieId")["rating"].values
    rvi = uv_ratings[uv_ratings["userId"] == v].sort_values(by="movieId")["rating"].values
    l2u = ratings.get_user_rating_norms(user=u)
    l2v = ratings.get_user_rating_norms(user=v)
    S[uv] = rui.dot(rvi) / (l2u * l2v)
similarity = pd.DataFrame.from_dict(
    data=S, orient="index", columns=["similarity"]
).reset_index()

similarity[["u", "v"]] = similarity["index"].str.split("_", 1, expand=True)


In [10]:
ratings = IOService.read(FILEPATH)
u_count = ratings.userId.nunique()
i_count = ratings.movieId.nunique()
row = ratings.userId.values
col = ratings.movieId.values
data = ratings.rating.values
csr = csr_matrix((data,(row,col)), shape=(u_count,i_count))

ValueError: row index exceeds matrix dimensions